In [ ]:
import streamlit as st

In [ ]:
st.markdown("""
<div style="background-color: #000020; color: white; text-align: center; padding: 20px">
  <h1 style="margin: 0; color: white"><b>Questioning the Answers: LLMs enter the Boardroom</b></h1>
  <h2 style="margin: 0; color: white"><b></b>Using Gen AI Tools to Harness Alpha from Earnings Calls</h2>
  <h3 style="margin: 0; color: white"><b>by S&P Global Market Intelligence's Quantitative Research & Solutions (QRS) Group</b></h3>
</div>
""", unsafe_allow_html=True)

# 1. Overview
Earnings calls play a pivotal role in shaping investor perceptions. The quality of communication between executives and analysts can significantly influence company performance. On-topic and proactive exeutives, who deliver proactive presentations, anticipate market queries, and provide clear, on-topic answers to analysts’ questions—consistently outperform their peers. Conversely, off-topic and reactive executives, who fail to address analysts’ key inquiries during presentations, and provide off-topic responses—significantly underperform.

Executives' ability to anticipate investor concerns and maintain a focused dialogue fosters confidence and strategic communication. In contrast, failing to provide clarity when analysts seek additional information can lead to misalignment and breakdowns in transparency. A long (short) portfolio of on-topic and proactive (off-topic and reactive) generates +515bps of annualized alpha.

This notebook serves as a introduction for the research detailed in Quantitative Research & Solutions’ recent publication, ["Questioninig the Answers: LLM's enter the Boardroom."](https://www.spglobal.com/market-intelligence/en/news-insights/research/questioning-the-answers-llms-enter-the-boardroom) IIt analyse executive on-topicness and proactiveness using the analysts questions, executives answers and LLM answers. This research harness alpha using LLM tools, including vector embeddings, vector cosine similarity, and the LLM quesiton answering. There is a longer version avalible upon request that also covers how to create the input data from the datasets described in section 2, please reach out to QRS@spglobal.com for access to the longer version.

# 2. Datasets

The ["Questioninig the Answers: LLM's enter the Boardroom."](https://www.spglobal.com/market-intelligence/en/news-insights/research/questioning-the-answers-llms-enter-the-boardroom) research is using the datasets below from the Snowflake Marketplace. Access to those are not neccessary for running this QuickStart, where we are using a sample datase.

To reproduce the full research using the complete datasets then request access to those below using the links or contact SnowflakeMarketplace@spglobal.com.

|Name|Description |
|----|----|
|[ S&P Capital IQ Financials](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D2N/s-p-global-market-intelligence-s-p-capital-iq-financials)|S&P Capital IQ Financials provides global standardized financial statement data for over 180,000 companies, including over 95,000 active and inactive public companies, and As Reported data for over 150,000 companies. S&P Capital IQ Standardized Financials allows you to extend the scope of your historical analysis and back-testing models with consistent data from all filings of a company's historical financial periods including press releases, original filings, and all restatements.|
|[Global Events](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D38/s-p-global-market-intelligence-global-events)|The Global Events dataset provides details on upcoming and past corporate events such as earnings calls, shareholder/analyst meetings, expected earnings release dates and more. With deep history back to 2003, clients can leverage this dataset to derive signals and support trading models across asset classes, trading styles and frequencies. This dataset also helps in research & analysis, risk management & compliance, and trade surveillance workflows.|
|[Machine Readable Transcripts](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D2V/s-p-global-market-intelligence-machine-readable-transcripts)|The Machine Readable Transcripts dataset aggregates data from earnings calls delivered in a machine-readable format for Natural Language Processing (NLP) applications with metadata tagging. Leverage Machine Readable Transcripts to keep track of event information for specific companies including dates, times, dial-in and replay numbers and investor relations contact information. Easily combine data from earnings, M&A, guidance, shareholder, company conference presentations and special calls with traditional datasets to develop proprietary analytics.|
|[Compustat® Financials](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D2R/s-p-global-market-intelligence-compustat®-financials)|Compustat Financials provides standardized North American and global financial statements and market data for over 80,000 active and inactive publicly traded companies that financial professionals have relied on for over 50 years. Compustat allows investment professionals, academic researchers, and industry analysts to combine deep history with robust and consistent data standardization into their research and backtesting to produce valuable insights and generate alpha. With historical data for North America as far back as 1950 and point-in-time snapshots beginning in 1987, Compustat provides you with insight into company financial performance across many different economic cycles not available anywhere else.|

# 3. Libraries & User Inputs
Import libraries required for the workflow

## 3.1 Libraries

Before running, mak sure you have added **cachetools** through **packages**

In [ ]:
# Import python packages
import json

from snowflake.snowpark import functions as snow_funcs
from snowflake.snowpark import Window

from snowflake.cortex import embed_text_768, complete, CompleteOptions, summarize

from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Mapping of functions that is not exposed in the Snowpark API
snf_ifnull = snow_funcs.function("IFNULL")
snf_vector_cosine_similarity = snow_funcs.function("vector_cosine_similarity")
snf_count_tokens = snow_funcs.function("SNOWFLAKE.CORTEX.COUNT_TOKENs")

## 3.2 User Inputs

This research invloves the usage of an embedding model and a completion model, the default models were set to "snowflake-arctic-embed-m" for embedding and "llama3.1-8b" for completion. This user input section gives you the flexibility to chose your own model for the task.

In [ ]:
# Which embedding model we want to use, see https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#availability 
# for avalible embedding models in Snowflake
embedding_model = "snowflake-arctic-embed-m" 

# Which LLM we want to use, see https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#availability 
# for avalible LLMs in Snowflake
completion_model = "llama3.1-8b" 

# Name of the databse created in the Setup Snowflake step
sp_llm_qs_location = "SP_LLM_QS.PUBLIC"


# 4. S&P Global Q4 2024 Earnings Call Transcript

The dataset we are going to use are the prepared remarks, questions and answers sections of the S&P Global Q4 2024 Earnings Call. They have been tokenized on sentence level, each row has one sentence in the PROCESSEDTEXT column and the COMPONENTTEXT has the full text for the prepared remarks, questions and answers.

In [ ]:
all_component_df = session.table(f"{sp_llm_qs_location}.SAMPLE_TRANSCRIPT")
all_component_df.sort("COMPONENTORDER", "SENTENCEORDER").limit(50)

# 5. Working with the Data: Retrival Augmented Generation (RAG)
Retrieval-Augmented Generation (RAG) is a tool that improves LLM consistency by retrieving relevant information before answering a question.

For this task the LLM needs to be as consistent as possible in its responses to the analysts’ questions as inconsistency will lead to variations in cosine similarity scores and disrupt feature generation downstream.

To combat this, we designed a Retrieval-Augmented Generation (RAG) engine that chunks the prepared remarks sentence-by-sentence and retrieves the optimal retrieval percentage of sentences most similar to the question. Inconsistency occurs when the LLM is provided with too little (or too much) context, it becomes uninformed (unspecific). The optimal retrieval percentage for consistency is 60%.

In the cells below, we vector embed all questions, prepared remark sentences and answer sentences using the snowflake-arctic-m embedding model. Then use the cosine similarity from the (question vs prepared remark sentences) and (question vs answer sentences) to select top 60% most relevant prepared remark and answer sentences to the question from S&P Global Q4 2024 Earnings Call Transcript

## 5.1 S&P Global Q4 2024 Earnings Call Transcript
Select and seperate the prepared remarks, questions and answers sections of the S&P Global Q4 2024 Earnings Call

In [ ]:
# Get all Prepared Remarks,
prepared_remarks = all_component_df.filter(snow_funcs.col('transcriptComponentTypeId') == 2) 
# Get all Analyst Questions
questions = all_component_df.filter(snow_funcs.col('transcriptComponentTypeId') == 3)
# Get all Answers
answers = all_component_df.filter(snow_funcs.col('transcriptComponentTypeId') == 4) 

st.dataframe(prepared_remarks.limit(5))
st.dataframe(questions.limit(5))
st.dataframe(answers.limit(5))

## 5.2 Vector Embedding
Transform the questions, prepared remark sentences and answer sentences into numerical
representations using the snowflake-arctic-m embedding model.

### 5.2.2 Apply Embedding to Transcript Components

In [ ]:
# Generate embeddings for the prepared remarks using the embed_text_768 function and store them in a new column, PrepRemarkSentenceVec
prepared_remarks_vec_df = (prepared_remarks
                            .withColumn("PrepRemarkSentenceVec"
                                        , embed_text_768(snow_funcs.lit(embedding_model),prepared_remarks["PROCESSEDTEXT"]))
                           ).cache_result()
# Generate embeddings for the questions using the embed_text_768 function and store them in a new column, questionVec
questions_vec_df = (questions
                        .withColumn("questionSentenceVec"
                                    , embed_text_768(snow_funcs.lit(embedding_model),questions["PROCESSEDTEXT"]))
                  ).cache_result()
# Generate embeddings for the answers using the embed_text_768 function and store them in a new column, sentenceVec
answers_vec_df = (answers
                    .withColumn("answerSentenceVec"
                                , embed_text_768(snow_funcs.lit(embedding_model),answers["PROCESSEDTEXT"]))
                        ).cache_result()

# 5.3 Cosine Similarity
To determine semantic closeness, we vector-embed the question-and-answer texts and calculate a cosine similarity score between the two vectors.
For example, A and B each represent a vector such that:


$$	A = [a_1,a_2,… a_n] $$
$$	B = [b_1,b_2,… b_n] $$


The cosine similarity formula between vectors A and B is:

$$\text{Cosine Similarity} = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \cdot \|\mathbf{B}\|}$$

, where A⋅B is the dot product of the vectors, and |A|⋅|B| is the product of each vector's magnitude.
The result ranges from -1 to 1, where:

	-1: Vectors are opposite.
	0: Vectors are unrelated.
	1: Vectors are identical.

### 5.3.2 Apply Cosine Similarity to Question & Answer Vector Embeddings
The cell below creates question and answer pair by collecting all the answer sentences whose componentOrder is between the currentQuestionComponentOrder and the nextQuestionComponentOrder. Then the the consine similarity was applied to the question and answer sentence vector embeddings.

In [ ]:
# Rename the columns so it's easier to use them when comparing 
# and add a new column that capture the id of the next question
questions_vec_rn_df = (questions_vec_df
                        .rename({"speakerTypeName": "questionSpeakerTypeName"
                                 , "transcriptPersonName": "questionTranscriptPersonName"
                                 , "transcriptPersonId":"questionTranscriptPersonId"
                                 , "proId": "questionProId"
                                 , "transcriptComponentTypeId": "questionTranscriptComponentTypeId"
                                 , "transcriptComponentId": "questionTranscriptComponentId"
                                 , "componentOrder": "currentQuestionOrder"
                                 , "processedText": "question"})
                        .with_column("nextQuestionOrder"
                                     ,snow_funcs.lead("currentQuestionOrder")
                                                    .over(Window.partition_by("tradingItemId", "transcriptId")
                                                                .order_by("currentQuestionOrder"))
                                    )
                    )
# Jon the questions and answers and get the cosine simularity between the questionVec
# and answerSentenceVec 
questions_answers_cos_df = (questions_vec_rn_df
        .join(answers_vec_df, ((questions_vec_rn_df.tradingItemId == answers_vec_df.tradingItemId)
                    & (questions_vec_rn_df.transcriptId == answers_vec_df.transcriptId)
                    & (answers_vec_df.componentOrder.between(questions_vec_rn_df.currentQuestionOrder
                                                                              , snf_ifnull(questions_vec_rn_df.nextQuestionOrder
                                                                                            , snow_funcs.lit(10000))))
                    )
                )
        .select(questions_vec_rn_df.calldate.as_("calldate") ,questions_vec_rn_df.enteredDate.as_("enteredDate")
                , questions_vec_rn_df.fiscalyearquarter.as_("fiscalyearquarter")
                , questions_vec_rn_df.calendarYearQuarter.as_("calendarYearQuarter")
                , questions_vec_rn_df.tradingItemId.as_("tradingItemId"), questions_vec_rn_df.companyId.as_("companyId")
                , questions_vec_rn_df.companyName.as_("companyName"), questions_vec_rn_df.headline.as_("headline")
                , questions_vec_rn_df.transcriptId.as_("transcriptId"), questions_vec_rn_df.questionSpeakerTypeName
                , questions_vec_rn_df.questionTranscriptPersonName, questions_vec_rn_df.questionTranscriptPersonId
                , questions_vec_rn_df.questionProId,  answers_vec_df.speakerTypeName.as_("answerSpeakerTypeName")
                , answers_vec_df.transcriptPersonName.as_("answerTranscriptPersonName")
                , answers_vec_df.transcriptPersonId.as_("answerTranscriptPersonId")
                , answers_vec_df.proId.as_("answerProId"), questions_vec_rn_df.questionTranscriptComponentTypeId
                , answers_vec_df.transcriptComponentTypeId.as_("answerTranscriptComponentTypeId")
                , questions_vec_rn_df.questionTranscriptComponentId
                , answers_vec_df.transcriptComponentId.as_("answerTranscriptComponentId")
                , questions_vec_rn_df.currentQuestionOrder, questions_vec_rn_df.nextQuestionOrder
                , answers_vec_df.componentOrder.as_("answerOrder")
                , answers_vec_df.sentenceOrder.as_("answerSentenceOrder")
                , questions_vec_rn_df.question, answers_vec_df.componentText.as_("answer")
                , answers_vec_df.processedText.as_("answerSentence")
                , questions_vec_rn_df.questionSentenceVec, answers_vec_df.answerSentenceVec
                , snf_vector_cosine_similarity(snow_funcs.col("questionSentenceVec")
                                               , snow_funcs.col("answerSentenceVec")).as_("questionAnswerCosSim")
               )
      .sort(questions_vec_rn_df.nextQuestionOrder.asc_nulls_last()
            , answers_vec_df.componentOrder.asc_nulls_last()
            , answers_vec_df.sentenceOrder.asc_nulls_last()
        )
     ).cache_result()

questions_answers_cos_df.limit(5)

### 5.3.3 Apply Cosine Similarity to Question & Prepared Remarks Vector Embeddings
The cell below pairs all prepared remarks sentences to questions. Then the the consine similarity was applied to the question and prepared remarks sentence vector embeddings.

In [ ]:
questions_prepared_remarks_cos_df = (questions_vec_rn_df
                    .join(prepared_remarks_vec_df, ((questions_vec_rn_df.tradingItemId == prepared_remarks_vec_df.tradingItemId)
                                                & (questions_vec_rn_df.transcriptId == prepared_remarks_vec_df.transcriptId)
                                            )
                    )
                  .select(questions_vec_rn_df.calldate.as_("calldate") ,questions_vec_rn_df.enteredDate.as_("enteredDate")
                          , questions_vec_rn_df.fiscalyearquarter.as_("fiscalyearquarter")
                          , questions_vec_rn_df.calendarYearQuarter.as_("calendarYearQuarter")
                          , questions_vec_rn_df.tradingItemId.as_("tradingItemId"), questions_vec_rn_df.companyId.as_("companyId")
                          , questions_vec_rn_df.companyName.as_("companyName"), questions_vec_rn_df.headline.as_("headline")
                          , questions_vec_rn_df.transcriptId.as_("transcriptId"), questions_vec_rn_df.questionSpeakerTypeName
                          , questions_vec_rn_df.questionTranscriptPersonName, questions_vec_rn_df.questionTranscriptPersonId
                          , questions_vec_rn_df.questionProId, questions_vec_rn_df.questionTranscriptComponentTypeId
                          , questions_vec_rn_df.questionTranscriptComponentId, questions_vec_rn_df.currentQuestionOrder
                          , questions_vec_rn_df.nextQuestionOrder, questions_vec_rn_df.question, questions_vec_rn_df.questionSentenceVec
                          , prepared_remarks_vec_df.componentOrder.as_("executiveRemarkComponentOrder")
                          , prepared_remarks_vec_df.sentenceOrder.as_("executiveRemarkSentenceOrder")
                          , prepared_remarks_vec_df.componentText.as_("executiveRemark")
                          , prepared_remarks_vec_df.processedText.as_("executiveSentence")
                          , prepared_remarks_vec_df.PrepRemarkSentenceVec.as_("executiveVec")
                          ,snf_vector_cosine_similarity(questions_vec_rn_df.questionSentenceVec
                                                        , prepared_remarks_vec_df.prepRemarkSentenceVec).as_("questionExecCosSim")
                         )
                  .sort(questions_vec_rn_df.nextQuestionOrder.asc_nulls_last()
                        , prepared_remarks_vec_df.componentOrder.asc_nulls_last()
                        , prepared_remarks_vec_df.sentenceOrder.asc_nulls_last())
                 ).cache_result()

questions_prepared_remarks_cos_df.limit(5)

## 5.4 Top 60% Sentences
Utilizing the top 60% of prepared remarks identified as generating the most consistent LLM output. For further details on the experiment, please refer to the 'LLM Robustness Check' section in the whitepaper.

### 5.4.2 Concat Top 60% Answer Sentences
After selecting the top 60% most similar answer sentences, we concat the answer sentences on the question level.

In [ ]:
# First, add two new columns where similarityRank is the row number within 
# each currentQuestionOrder order by the cosin similarity between question sentences 
# and answer sentences, second add a flag column,toKeepFlag, with a 1 
# if the sentence are part of top 60% 
qa_sim_rank_df = (questions_answers_cos_df
                       .with_columns(["similarityRank"
                                      , "answerSentencesCount"]
                                    ,[snow_funcs.row_number().over(Window.partition_by("tradingItemId", "transcriptId"
                                                                                       , "currentQuestionOrder")
                                                                        .order_by(snow_funcs.col("questionAnswerCosSim").desc())
                                                                  )
                                      ,snow_funcs.count('*').over(Window.partition_by("tradingItemId", "transcriptId"
                                                                                      , "currentQuestionOrder")
                                                                 )
                                    ])
                        .with_column("toKeepFlag"
                                    , snow_funcs.when(snow_funcs.col("answerSentencesCount") == 1
                                                      , snow_funcs.lit(1))
                                                .when(snow_funcs.col("similarityRank") <= 
                                                                    snow_funcs.col("answerSentencesCount") * 0.67
                                                      , snow_funcs.lit(1))
                                                .otherwise(snow_funcs.lit(0))
                                    )
                      )

# Filter out Top 60% sentences, concatinate them and count the tokens for answer and sixtyPercentAnswer
q_w_sixty_perc_a_df = (qa_sim_rank_df
                        .filter(qa_sim_rank_df.toKeepFlag == 1)
                        .group_by("callDate", "enteredDate", "fiscalYearQuarter", "calendarYearQuarter"
                                  , "tradingItemId", "companyId", "companyName", "headline", "transcriptId"
                                  , "questionSpeakerTypeName", "questionTranscriptPersonName", "questionTranscriptPersonId"
                                  , "questionProId", "answerSpeakerTypeName", "answerTranscriptPersonName"
                                  , "answerTranscriptPersonId", "answerProId", "questionTranscriptComponentTypeId"
                                  , "answerTranscriptComponentTypeId", "questionTranscriptComponentId"
                                  , "answerTranscriptComponentId", "currentQuestionOrder", "nextQuestionOrder"
                                  , "answerOrder", "question", "answer")
                       .agg(snow_funcs.array_to_string(snow_funcs.array_agg(snow_funcs.col("answerSentence"))
                                                        , snow_funcs.lit(' ')).as_("sixtyPercentAnswer"))
                       .with_columns(["answerTokenCount"
                                     , "sixtyPercentAnswerTokenCount"]
                                     ,[snf_count_tokens(snow_funcs.lit(completion_model),snow_funcs.col("answer"))
                                      , snf_count_tokens(snow_funcs.lit(completion_model),snow_funcs.col("sixtyPercentAnswer"))]
                           )
                      
                      ).cache_result()
q_w_sixty_perc_a_df.limit(5)

### 5.4.3 Concat Top 60% Prepared Remarks Sentences
Similarly, after selecting the top 60% most similar prepared remarks sentences, we concat the prepared remarks sentences on the question level.

In [ ]:
# First, add two new columns where similarityRank is the row number within each currentQuestionOrder 
# order by the cosin similarity between question sentences  and answer sentences, second add a flag column,toKeepFlag, with a 1 
# if the sentence are part of top 60% 
exec_questions_rank_df = (questions_prepared_remarks_cos_df
                                .with_columns(["similarityRank"
                                               , "executiveRemarksSentencesCount"]
                                              ,[snow_funcs.row_number().over(Window.partition_by("tradingItemId"
                                                                                                 , "transcriptId"
                                                                                                 , "currentQuestionOrder")
                                                                                    .order_by(snow_funcs.col("questionExecCosSim").desc_nulls_last()))
                                                , snow_funcs.count('*').over(Window.partition_by("tradingItemId", "transcriptId"
                                                                                                 , "currentQuestionOrder"))])
                                .with_column("toKeepFlag"
                                     , snow_funcs.when(snow_funcs.col("executiveRemarksSentencesCount") == 1
                                                       , snow_funcs.lit(1))
                                        .when(snow_funcs.col("similarityRank") <= snow_funcs.col("executiveRemarksSentencesCount") * 0.67
                                                        , snow_funcs.lit(1))
                                        .otherwise(snow_funcs.lit(0)))
                                 )

# Filter out Top 60% sentences and concatinate them and recreate the full opening remarks
exec_questions_sixty_perc_df = (exec_questions_rank_df
                                .filter(exec_questions_rank_df.toKeepFlag == 1)
                                .group_by("callDate", "enteredDate", "fiscalYearQuarter", "calendarYearQuarter"
                                          , "tradingItemId", "companyId", "companyName", "headline", "transcriptId"
                                          , "questionSpeakerTypeName", "questionTranscriptPersonName"
                                          , "questionTranscriptPersonId", "questionProId"
                                          , "questionTranscriptComponentTypeId", "questionTranscriptComponentId"
                                          , "currentQuestionOrder", "nextQuestionOrder", "executiveRemarkComponentOrder"
                                          , "question")
                                        .agg(snow_funcs.listagg(snow_funcs.col("executiveSentence"), ' ').as_('sixtyPercentExecutiveRemark')
                                            ,snow_funcs.listagg(snow_funcs.col("executiveRemark"), '\n\n').as_('fullExecutiveRemark'))
                                   ).cache_result()

exec_questions_sixty_perc_df

# 6. Working with the Data: LLM Ready Data

Using a LLM to answer analysts questions based only on the prepared remarks and the previous questions and answers will give an indication if executives are proactive.

## 6.1 Using Snowflake Cortex AI
When calling the Snowflake Cortex COMPLETE function, messages are organized into distinct roles—system, user, and assistant—to structure and guide interactions. Each role serves a specific purpose:

System: Provides instructions that define the context or behavior of the model. It's like setting the rules or tone for the conversation. Example: "You are a helpful assistant that answers questions about technology in a concise manner."

User: Represents the input or queries made by the person interacting with the model. These are the prompts or requests that the model responds to. Example: "What is the purpose of the OpenAI API?"

Assistant: Reflects the model's response to the user's query, shaped by the system's instructions and the user's input. Example: "The OpenAI API is designed to enable developers to integrate language models into their applications for tasks like answering questions, generating content, and more."

In our research, executive prepared remarks are labelled as assistant messages, analyst's questions as User messages and executive answers as Assistant messages

### 6.1.1 Construct COMPLETE Assistant Message with QA Pair Snippet

Create a object that has the Analyst question with the role as user and the answer with the role as assitsant example:
```
[
  {
    "content": "Analyst Question",
    "role": "user"
  },
  {
    "content": "Executive 60% Answer",
    "role": "assistant"
  }
]
```

In [ ]:

q_w_sixty_perc_a_prompt_df = (q_w_sixty_perc_a_df
                                     .with_columns(["questionPromptSnippet", "answerPromptSnippet"]
                                                  ,[snow_funcs.object_construct(
                                                            snow_funcs.lit('role'), snow_funcs.lit('user')
                                                            , snow_funcs.lit('content')
                                                            , snow_funcs.replace(q_w_sixty_perc_a_df.question
                                                                                 , snow_funcs.lit('\r'), snow_funcs.lit(''))
                                                        )
                                                        , snow_funcs.object_construct(
                                                            snow_funcs.lit('role'), snow_funcs.lit('assistant')
                                                            , snow_funcs.lit('content')
                                                            , snow_funcs.replace(q_w_sixty_perc_a_df.sixtyPercentAnswer
                                                                                , snow_funcs.lit('\r'), snow_funcs.lit(''))
                                                        )])
                             ).cache_result()
#transComp_pppQPairTop60AnswerConcat_df
q_w_sixty_perc_a_prompt_df

### 6.1.2 Construct Snowflake Cortex COMPLETE Assistant Message with Prepared Remarks Snippets

Create a object that has the Analyst question with the role as user and the 60% prepared remarks with the role as assitsant example:
```
[
  {
    "content": "Analyst Question",
    "role": "user"
  },
  {
    "content": "60% Prepared Remarks",
    "role": "assistant"
  }
]
```

In [ ]:
exec_questions_sixty_perc_prompt_df = (exec_questions_sixty_perc_df
                                     .with_columns(["questionPromptSnippet"
                                                   , "remarksPromptSnippet"]
                                                  , [snow_funcs.object_construct(
                                                            snow_funcs.lit('role'), snow_funcs.lit('user')
                                                            , snow_funcs.lit('content')
                                                            , snow_funcs.replace(exec_questions_sixty_perc_df.question
                                                                                 , snow_funcs.lit('\r'), snow_funcs.lit(''))
                                                        )
                                                     , snow_funcs.object_construct(
                                                            snow_funcs.lit('role'), snow_funcs.lit('assistant')
                                                            , snow_funcs.lit('content')
                                                            , snow_funcs.replace(exec_questions_sixty_perc_df.sixtyPercentExecutiveRemark
                                                                                , snow_funcs.lit('\r'), snow_funcs.lit(''))
                                                        )
                                                    ])
                                            ).cache_result()
exec_questions_sixty_perc_prompt_df.limit(5)

## 6.2 Collect All Messages for Cortex COMPLETE
All question pairs with prepare remarks and answers come together to form an LLM prompt following the iterative process such that:

1. 'user': 'From the perspective of a top executive, please answer the following question raised by a financial analyst during an earnings conference call. Knowledge cutoff date: '  
2. 'assistant': 60% prepared remarks  
3. 'user': question 1  
4. 'assistant': 60% answer 1  
5. ...  
6. ...  
7. 'user': question n  

### 6.2.1 LLM Ready Prompt Messages
In the dataframe below, the prompt column has all the messages in 1 list. This is the prompt for the LLM.

In [ ]:
#transComp_pppQPairTop60AnswerConcat_df -> q_w_sixty_perc_a_prompt_df

q_w_sixty_perc_a_LLM_promp_df = (q_w_sixty_perc_a_prompt_df
                                .with_columns(["initPrompt", "concatenatedPredecessors"]
                                             ,[snow_funcs.array_construct(
                                                    snow_funcs.object_construct(snow_funcs.lit('role'), snow_funcs.lit('user')
                                                                               , snow_funcs.lit('content')
                                                                               , snow_funcs.concat(snow_funcs.lit('From the perspective of a top executive, please answer the following question raised by a financial analyst during an earnings conference call. Knowledge cutoff date: ')
                                                                                                  , snow_funcs.to_char(exec_questions_sixty_perc_prompt_df.callDate))))
                                                 
                                             , snow_funcs.array_flatten(
                                                 snow_funcs.array_agg(snow_funcs.array_construct(q_w_sixty_perc_a_prompt_df.questionPromptSnippet
                                                                                                 , q_w_sixty_perc_a_prompt_df.answerPromptSnippet))
                                                                .over(Window.partition_by(q_w_sixty_perc_a_prompt_df.transcriptid)
                                                                            .order_by(q_w_sixty_perc_a_prompt_df.CURRENTQUESTIONORDER)
                                                                            .rows_between(Window.unboundedPreceding, -1))
                                                        
                                                )]
                                )
                                .with_column("prompt"
                                            , snow_funcs.array_cat(snow_funcs.col("initPrompt")
                                                                  ,snow_funcs.array_cat(snow_funcs.col("concatenatedPredecessors")
                                                                                       , snow_funcs.array_construct(snow_funcs.col("questionPromptSnippet")))
                                                                  ))
                            ).cache_result()
q_w_sixty_perc_a_LLM_promp_df

## 6.3 Collect LLM Response

### 6.3.1 Apply LLM Completion
We apply the SNOWFLAKE.CORTEX.COMPLETE function on the prompt column using the model defined by `completion_model`and collect the LLM response.

In [ ]:
opts = CompleteOptions(temperature = 0)

question_LLM_answer_raw_df = (q_w_sixty_perc_a_LLM_promp_df
                                    .with_column("LLMAnswer"
                                                , complete(model=completion_model
                                                           , prompt=q_w_sixty_perc_a_LLM_promp_df.prompt
                                                          , options=opts))
                               ).cache_result()

question_LLM_answer_raw_df.limit(5)

### 6.3.2 Clean Up LLM Response

Extract only the actual message from the LLM from the responses

In [ ]:
question_LLM_answer_df = (question_LLM_answer_raw_df
                                        .with_column("cleanLLMAnswer"
                                         , snow_funcs.to_varchar(question_LLM_answer_raw_df.LLMANSWER['choices'][0]['messages']))
                                      .select("callDate", "tradingItemId", "transcriptId", "headline"
                                              , "questionTranscriptPersonName", "questionTranscriptPersonId"
                                              , "questionProId", "answerTranscriptPersonName", "answerTranscriptPersonId"
                                              , "answerProId","questionTranscriptComponentId", "answerTranscriptComponentId"
                                              , "question", "answer", "cleanLLMAnswer")
                                     ).cache_result()

question_LLM_answer_df.limit(5)

## 6.4 Summarize Text

Use the Snowflake Cortex Summarize function to summarize the question, answer and LLM answer. This is done so we can compare them later. 

In [ ]:
question_LLM_answer_summarize_df = (question_LLM_answer_df
                                            .with_columns(["summarizeQuestion", "summarizeAnswer", "summarizeCleanLLMAnswer"]
                                                         ,[summarize(question_LLM_answer_df.question)
                                                          ,summarize(question_LLM_answer_df.answer)
                                                           ,summarize(question_LLM_answer_df.cleanLLMAnswer)
                                                          ]
                                                    )
                                         ).cache_result()

question_LLM_answer_summarize_df.limit(5)

# 7. Working with the Data: Factor Construction
## 7.1 Executive On/Off Topic Factor
When an executive answer is semantically similar (dissimilar) to the analyst’s question, it suggests that the answer uses language and concepts similar to (different from) the analyst question, indicating it is on-topic (off-topic). To determine semantic closeness, we vector-embed the question-and-answer texts and calculate a cosine similarity score between the two vectors.

### 7.1.1 Question vs Executive Answer Cosine Similarity

In [ ]:
qa_exec_on_off_topic_factor_df = (question_LLM_answer_summarize_df
                                        .with_columns(["questionVec", "answerVec"]
                                                    ,[embed_text_768(embedding_model, question_LLM_answer_summarize_df.summarizeQuestion)
                                                     ,embed_text_768(embedding_model, question_LLM_answer_summarize_df.summarizeAnswer)])
                                        .with_column("execOnOffTopicFactor"
                                                    , snf_vector_cosine_similarity(snow_funcs.col("questionVec"), snow_funcs.col("answerVec")))
                                    ).cache_result()
qa_exec_on_off_topic_factor_df.limit(5)

### 7.1.2 Transcript Mean Executive On/Off Topic Factor
Cosine similarity scores are averaged at the transcript level. A high (low) Cosine Similarity Score indicates an On (Off) Topic Executive.



In [ ]:
qa_exec_on_off_topic_factor_df.select(snow_funcs.avg("execOnOffTopicFactor").as_("transcriptLevelExecOnOffTopicFactor"))

## 7.2 Executive Proactive/Reactive Factor
### 7.2.1 Question vs LLM Answer Cosine Similarity
Since the LLM answers only within the context of information provided in the prepared remarks, a high (low) cosine similarity score indicates that the LLM answers are semantically similar (dissimilar) the questions, reflecting the executives are proactive (reactive).

In [ ]:
q_exec_proactive_reactive_factor_df = (question_LLM_answer_summarize_df
                                        .with_columns(["questionVec", "LLMAnswerVec"]
                                                    ,[embed_text_768(embedding_model, question_LLM_answer_summarize_df.summarizeQuestion)
                                                    ,embed_text_768(embedding_model, question_LLM_answer_summarize_df.summarizeCleanLLMAnswer)])
                                        .with_column("execProactiveReactiveFactor"
                                                    , snf_vector_cosine_similarity(snow_funcs.col("questionVec"), snow_funcs.col("LLMAnswerVec")))
                                    ).cache_result()
q_exec_proactive_reactive_factor_df.limit(5)

### 7.2.2 Transcript Mean Executive Proactive/Reactive Factor
Similar to the construction of the Executive On/Off Topic factor, both the LLM answers and questions are summarized, vector-embedded and cosine similarity scores are averaged at the transcript level.

In [ ]:
q_exec_proactive_reactive_factor_df.select(snow_funcs.avg("execProactiveReactiveFactor").as_("transcriptLevelexecProactiveReactiveFactor"))

# 8. Ask your own questions

Based on on the prepared remarks, the questions and answers during the call you can use Cortex Complete with one of the included LLMs to ask your own questions. The code below will use the existing Questions and Answers and then append your question to it and use it with the choosen LLM.

In [ ]:
message_history = json.loads(question_LLM_answer_raw_df
                             .sort(question_LLM_answer_raw_df.questionTranscriptComponentId.desc_nulls_last()
                                   , question_LLM_answer_raw_df.answerTranscriptComponentId.desc_nulls_last())
                             .limit(1)
                             .select("prompt", "sixtyPercentAnswer")
                             .with_column("prompt_new"
                                          , snow_funcs.array_append(snow_funcs.col("prompt")
                                                                    ,snow_funcs.object_construct(snow_funcs.lit('role'), snow_funcs.lit('assistant')
                                                                                                 , snow_funcs.lit('content')
                                                                                                 , snow_funcs.replace(question_LLM_answer_raw_df.sixtyPercentAnswer
                                                                                                                      , snow_funcs.lit('\r'), snow_funcs.lit('')))
                                                                   )
                            )
                             .collect()[0]['PROMPT_NEW'])

st.write(f"Using model: {completion_model}")

#Generate a response
user_input_question = st.text_input("Ask me a question")

ask= st.button("Ask", key = "button_ask")
if ask: 
    message_history.append({'role':'user', 'content': user_input_question})
    data = complete(model=completion_model, prompt=message_history, session=session)
    with st.chat_message("model", avatar ="assistant"):
        st.write(data)

# 9. Results & Summary
This research underscores the significant impact of executive communication styles during earnings calls on firm performance. Proactive executives who anticipate market concerns and provide concise, on-topic responses foster transparency, aligning with investor expectations and driving superior returns. The findings demonstrate that firms with Efficient Communicators achieve statistically significant outperformance, while Total Redirectors suffer from diminished confidence and underperformance. These insights validate the critical role of strategic communication in shaping investor perceptions and influencing market outcomes.

Advanced analytical tools, such as vector embeddings and cosine similarity metrics, enable nuanced evaluations of executive-analyst interactions, revealing measurable performance effects across different communication styles. While large language models (LLMs) enhance feature extraction, challenges like forward-looking bias and inconsistency highlight the need for caution in time-sensitive tasks. Overall, the integration of proactive, clear, and relevant communication strategies remains paramount in fostering investor trust and maximizing financial success in a competitive marketplace.